In [1]:
import os

if "_executed_once" not in get_ipython().user_ns:
    print("This cell runs only once per kernel restart.")
    get_ipython().user_ns["_executed_once"] = True

    import nest_asyncio

    nest_asyncio.apply()

    os.chdir("../")


This cell runs only once per kernel restart.


In [3]:
from pydantic_ai import Agent
from pydantic_ai.providers.ollama import OllamaProvider
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.mcp import MCPServerStreamableHTTP


# ollama_model = OpenAIChatModel(
#     model_name="qwen3:8b",
#     provider=OllamaProvider(base_url="http://localhost:11434/v1"),
# )

ollama_model = OpenAIChatModel(
    model_name="gpt-oss:120b-cloud",
    provider=OllamaProvider(base_url="https://ollama.com/v1",
                            api_key=os.getenv("OLLAMA_API_KEY")),
)

server = MCPServerStreamableHTTP("http://localhost:8001/mcp")
system_prompt= """You are a helpful assistant providing information based one the Internet Yellow Pages (IYP, a knowdledge graph about the Internet) documentation.
Use available tools to retrieve informations. Always assume the answer to the user request is in the documentation (it's your job to find it)"""
agent = Agent(ollama_model, toolsets=[server], system_prompt=system_prompt)

result = agent.run_sync(
    "Find the dataset associated to IXPs and show me how IXPs are modeled in IYP"
)
print(result.output)


**IXPs in IYP – the dataset and the data model**

---

## 1️⃣  Which dataset holds the IXP information?

The Internet Yellow Pages (IYP) gets authoritative IXP data from **PeeringDB**.  
The specific IYP dataset that you can query is:

```
dataset://peeringdb.ix
```

(You can also look at the related *fac* and *net* datasets for facilities and member networks.)

---

## 2️⃣  How are IXPs modeled in the IYP knowledge‑graph?

The PeeringDB crawler creates a rich graph that captures **IXPs, their facilities, LANs, member ASes, organisations and auxiliary data**.  
Below is a concise summary of the node types and the relationships that appear for an IXP.

| **Node type** | **Key property** | **What it represents** |
|---------------|------------------|------------------------|
| `(:IXP)` | `name` | The Internet Exchange Point itself |
| `(:Facility)` | `name` | Co‑location/data‑center where the IXP is housed |
| `(:Country)` | `country_code` | Country of the IXP (or facility) |
| `(:Organi

In [6]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="You are a helpful assistant providing information based one the Internet Yellow Pages (IYP, a knowdledge graph about the Internet) documentation.\nUse available tools to retrieve informations. Always assume the answer to the user request is in the documentation (it's your job to find it)", timestamp=datetime.datetime(2025, 11, 21, 10, 12, 30, 874414, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Find the dataset associated to IXPs and show me how IXPs are modeled in IYP', timestamp=datetime.datetime(2025, 11, 21, 10, 12, 30, 874420, tzinfo=datetime.timezone.utc))], run_id='2d8c9135-3f77-424a-b1f7-0947180f01c0'),
 ModelResponse(parts=[ThinkingPart(content="We need to find dataset associated to IXPs and then describe modeling of IXPs in IYP. Use list_datasets then get_resource for dataset about IXPs. Let's list datasets.", id='reasoning', provider_name='ollama'), ToolCallPart(tool_name='list_datasets', args='{}', tool_call_id='call

In [7]:
from typing import Any
from datetime import datetime
import json


def pretty_print_messages(messages: list[Any], verbose: bool = False) -> None:
    """
    Pretty print a list of Pydantic AI messages.

    Args:
        messages: List of ModelRequest and ModelResponse objects
        verbose: If True, show additional metadata like timestamps, usage, etc.
    """
    for i, msg in enumerate(messages):
        print(f"\n{'=' * 80}")
        print(f"Message {i + 1}")
        print(f"{'=' * 80}")

        # Determine message type
        if hasattr(msg, "kind"):
            if msg.kind == "request":
                _print_request(msg, verbose)
            elif msg.kind == "response":
                _print_response(msg, verbose)
        else:
            print(f"Unknown message type: {type(msg)}")


def _print_request(msg: Any, verbose: bool) -> None:
    """Print a ModelRequest message."""
    print("📤 REQUEST")

    if verbose and hasattr(msg, "instructions") and msg.instructions:
        print(f"\nInstructions: {msg.instructions}")

    print("\nParts:")
    for j, part in enumerate(msg.parts):
        _print_request_part(part, j, verbose)


def _print_response(msg: Any, verbose: bool) -> None:
    """Print a ModelResponse message."""
    print("📥 RESPONSE")

    if verbose:
        if hasattr(msg, "model_name") and msg.model_name:
            print(f"Model: {msg.model_name}")
        if hasattr(msg, "provider_name") and msg.provider_name:
            print(f"Provider: {msg.provider_name}")
        if hasattr(msg, "timestamp"):
            print(f"Timestamp: {msg.timestamp}")
        if hasattr(msg, "usage"):
            print(f"Usage: {msg.usage}")
        if hasattr(msg, "provider_request_id") and msg.provider_request_id:
            print(f"Request ID: {msg.provider_request_id}")
        if hasattr(msg, "provider_details") and msg.provider_details:
            print(f"Provider Details: {msg.provider_details}")
        if hasattr(msg, "finish_reason") and msg.finish_reason:
            print(f"Finish Reason: {msg.finish_reason}")

    print("\nParts:")
    for j, part in enumerate(msg.parts):
        _print_response_part(part, j, verbose)


def _print_request_part(part: Any, index: int, verbose: bool) -> None:
    """Print a single ModelRequestPart."""
    part_kind = getattr(part, "part_kind", "unknown")

    print(f"\n  [{index}] {_get_part_icon(part_kind)} {part_kind.upper()}")

    if part_kind == "system-prompt":
        print(f"      Content: {_truncate(part.content, 500)}")
        if verbose:
            if hasattr(part, "timestamp"):
                print(f"      Timestamp: {part.timestamp}")
            if hasattr(part, "dynamic_ref") and part.dynamic_ref:
                print(f"      Dynamic Ref: {part.dynamic_ref}")

    elif part_kind == "user-prompt":
        content = part.content
        if isinstance(content, str):
            print(f"      Content: {_truncate(content, 500)}")
        elif isinstance(content, list):
            print(f"      Content (multipart):")
            for item in content:
                _print_content_item(item)
        if verbose and hasattr(part, "timestamp"):
            print(f"      Timestamp: {part.timestamp}")

    elif part_kind == "tool-return":
        print(f"      Tool: {part.tool_name}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        print(f"      Content: {_format_json(part.content, indent=8)}")
        if verbose:
            if hasattr(part, "timestamp"):
                print(f"      Timestamp: {part.timestamp}")
            if hasattr(part, "metadata") and part.metadata:
                print(f"      Metadata: {part.metadata}")

    elif part_kind == "retry-prompt":
        print(f"      Tool: {part.tool_name or 'N/A'}")
        print(f"      Tool Call ID: {part.tool_call_id or 'N/A'}")
        if isinstance(part.content, str):
            print(f"      Content: {_truncate(part.content, 500)}")
        else:
            print(f"      Content (errors): {len(part.content)} validation errors")
            if verbose:
                for error in part.content[:3]:  # Show first 3 errors
                    print(f"        - {error}")
        if verbose and hasattr(part, "timestamp"):
            print(f"      Timestamp: {part.timestamp}")


def _print_response_part(part: Any, index: int, verbose: bool) -> None:
    """Print a single ModelResponsePart."""
    part_kind = getattr(part, "part_kind", "unknown")

    print(f"\n  [{index}] {_get_part_icon(part_kind)} {part_kind.upper()}")

    if part_kind == "text":
        print(f"      Content: {_truncate(part.content, 500)}")

    elif part_kind == "thinking":
        print(f"      Content: {_truncate(part.content, 500)}")
        if verbose:
            if hasattr(part, "id") and part.id:
                print(f"      ID: {part.id}")
            if hasattr(part, "signature") and part.signature:
                print(f"      Signature: {part.signature}")

    elif part_kind == "tool-call":
        print(f"      Tool: {part.tool_name}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        args = part.args
        if isinstance(args, str):
            print(f"      Args (JSON): {_truncate(args, 300)}")
        else:
            print(f"      Args: {_format_json(args, indent=8)}")

    elif part_kind == "builtin-tool-call":
        print(f"      Tool: {part.tool_name} (builtin)")
        print(f"      Provider: {part.provider_name or 'N/A'}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        args = part.args
        if isinstance(args, str):
            print(f"      Args (JSON): {_truncate(args, 300)}")
        else:
            print(f"      Args: {_format_json(args, indent=8)}")

    elif part_kind == "builtin-tool-return":
        print(f"      Tool: {part.tool_name} (builtin)")
        print(f"      Provider: {part.provider_name or 'N/A'}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        print(f"      Content: {_format_json(part.content, indent=8)}")
        if verbose:
            if hasattr(part, "timestamp"):
                print(f"      Timestamp: {part.timestamp}")
            if hasattr(part, "metadata") and part.metadata:
                print(f"      Metadata: {part.metadata}")


def _print_content_item(item: Any) -> None:
    """Print a content item from multipart user prompt."""
    if isinstance(item, str):
        print(f"        - Text: {_truncate(item, 200)}")
    elif hasattr(item, "kind"):
        kind = item.kind
        if kind == "image-url":
            print(f"        - Image URL: {item.url}")
        elif kind == "video-url":
            print(f"        - Video URL: {item.url}")
        elif kind == "audio-url":
            print(f"        - Audio URL: {item.url}")
        elif kind == "document-url":
            print(f"        - Document URL: {item.url}")
        elif kind == "binary":
            media = item.media_type
            size = len(item.data) if hasattr(item, "data") else 0
            print(f"        - Binary: {media} ({size} bytes)")
    elif hasattr(item, "type"):
        print(f"        - {item.type}: {item}")
    else:
        print(f"        - Unknown: {type(item)}")


def _get_part_icon(part_kind: str) -> str:
    """Get an emoji icon for the part kind."""
    icons = {
        "system-prompt": "⚙️",
        "user-prompt": "👤",
        "tool-return": "🔧",
        "retry-prompt": "🔄",
        "text": "💬",
        "thinking": "🤔",
        "tool-call": "🛠️",
        "builtin-tool-call": "🏗️",
        "builtin-tool-return": "🏗️",
    }
    return icons.get(part_kind, "❓")


def _truncate(text: str, max_length: int = 500) -> str:
    """Truncate text to a maximum length."""
    if len(text) <= max_length:
        return text
    return text[:max_length] + "..."


def _format_json(obj: Any, indent: int = 0) -> str:
    """Format an object as JSON with proper indentation."""
    if obj is None:
        return "null"

    try:
        if isinstance(obj, str):
            # Try to parse as JSON if it's a string
            try:
                parsed = json.loads(obj)
                return json.dumps(parsed, indent=2).replace("\n", "\n" + " " * indent)
            except:
                return obj
        else:
            return json.dumps(obj, indent=2, default=str).replace(
                "\n", "\n" + " " * indent
            )
    except:
        return str(obj)


In [8]:
pretty_print_messages(result.all_messages())


Message 1
📤 REQUEST

Parts:

  [0] ⚙️ SYSTEM-PROMPT
      Content: You are a helpful assistant providing information based one the Internet Yellow Pages (IYP, a knowdledge graph about the Internet) documentation.
Use available tools to retrieve informations. Always assume the answer to the user request is in the documentation (it's your job to find it)

  [1] 👤 USER-PROMPT
      Content: Find the dataset associated to IXPs and show me how IXPs are modeled in IYP

Message 2
📥 RESPONSE

Parts:

  [0] 🤔 THINKING
      Content: We need to find dataset associated to IXPs and then describe modeling of IXPs in IYP. Use list_datasets then get_resource for dataset about IXPs. Let's list datasets.

  [1] 🛠️ TOOL-CALL
      Tool: list_datasets
      Tool Call ID: call_8gnx9bue
      Args (JSON): {}

Message 3
📤 REQUEST

Parts:

  [0] 🔧 TOOL-RETURN
      Tool: list_datasets
      Tool Call ID: call_8gnx9bue
      Content: [
          {
            "organization": "Alice-LG",
            "name": "